In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [39]:
cv=LeaveOneOut()

In [40]:
#X = np.loadtxt("1.txt")
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [41]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    param1=["best","random"]
    param_grid={"splitter":param1}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["splitter"]
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
    param_grid={"max_depth":param2}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["max_depth"]
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,50))  ## 设置max_features范围
    param_grid={"max_features":param3}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
#     print(datetime.datetime.now())
#     print("best parameters:{}".format(grid_search.best_params_))                        #输出最优网格化参数
#     print("best cross-validation score:{:.3f}".format(grid_search.best_score_))         #成员提供优化过程中观察到的最好评分
#     print("best estimator:\n{}".format(grid_search.best_estimator_))
    Predict_Y=[]    #存放预测值
    Prob_Y=[]       #存放预测概率
    Really_Y=y  #存放真实标签
    for train_index,test_index in cv.split(X):
        estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=grid_search.best_params_["max_features"])
        estimator.fit(X[train_index],y[train_index])
        Predict_Y.append(estimator.predict(X[test_index])[0])
        Prob_Y.append(estimator.predict_proba(X[test_index]))
    Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
    Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
    Prob_Y=pd.DataFrame(Prob_Y)
    Prob_Y.columns=["0","1"]
    confusion_matrix(Really_Y,Predict_Y)
    TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
    FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
    FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
    TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
    ACC=accuracy_score(Really_Y,Predict_Y)
    F1_score=f1_score(Really_Y,Predict_Y)
    AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
    Recall=recall_score(Really_Y,Predict_Y)
    Precision=precision_score(Really_Y,Predict_Y)
    S=p1
    D=p2
    M=grid_search.best_params_["max_features"]
#     print("ACC:",ACC)
#     print("f1_score:",F1_score)
#     print("AUC:",AUC)
    return TN,FN,FP,TP,ACC,Recall,Precision,F1_score,AUC,S,D,M

In [33]:
file_path = 'E:\datatest\PseDPC\d2\d21' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
211.csv
**************************************************
2022-02-13 21:46:28.732639
2022-02-13 21:46:30.266533
2022-02-13 21:46:37.546056
['211.csv' '82' '13' '6' '72' '0.8901734104046243' '0.8470588235294118'
 '0.9230769230769231' '0.8834355828220859' '0.875668449197861' 'best' '3'
 '74']
##################################################
212.csv
**************************************************
2022-02-13 21:48:29.471600
2022-02-13 21:48:30.995523
2022-02-13 21:48:38.049680
['212.csv' '77' '10' '11' '75' '0.8786127167630058' '0.8823529411764706'
 '0.872093023255814' '0.8771929824561403' '0.8378342245989305' 'best' '3'
 '66']
##################################################
213.csv
**************************************************
2022-02-13 21:50:27.174686
2022-02-13 21:50:28.850204
2022-02-13 21:50:36.857785
['213.csv' '77' '8' '11' '77' '0.8901734104046243' '0.9058823529411765'
 '0.875' '0.8901734104046243' '0.817312834224598

In [42]:
file_path = 'E:\datatest\PseDPC\d2\d22' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
221.csv
**************************************************
2022-02-13 23:32:21.681913
2022-02-13 23:32:23.268667
2022-02-13 23:32:30.339771
['221.csv' '67' '17' '21' '68' '0.7803468208092486' '0.8'
 '0.7640449438202247' '0.7816091954022989' '0.8416443850267379' 'best' '3'
 '210']
##################################################
222.csv
**************************************************
2022-02-13 23:32:33.820437
2022-02-13 23:32:35.272579
2022-02-13 23:32:42.015538
['222.csv' '70' '15' '18' '70' '0.8092485549132948' '0.8235294117647058'
 '0.7954545454545454' '0.8092485549132947' '0.8276737967914438' 'best' '3'
 '210']
##################################################
223.csv
**************************************************
2022-02-13 23:32:45.563047
2022-02-13 23:32:46.957319
2022-02-13 23:32:53.773053
['223.csv' '71' '15' '17' '70' '0.815028901734104' '0.8235294117647058'
 '0.8045977011494253' '0.813953488372093' '0.7977272727272

In [34]:
file_path = 'E:\datatest\PseDPC\d2\d23' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
231.csv
**************************************************
2022-02-13 22:20:28.667111
2022-02-13 22:20:30.183089
2022-02-13 22:20:37.808653
['231.csv' '79' '10' '9' '75' '0.8901734104046243' '0.8823529411764706'
 '0.8928571428571429' '0.8875739644970414' '0.8826203208556149' 'best' '5'
 '93']
##################################################
232.csv
**************************************************
2022-02-13 22:23:00.456001
2022-02-13 22:23:01.875231
2022-02-13 22:23:08.643103
['232.csv' '72' '7' '16' '78' '0.8670520231213873' '0.9176470588235294'
 '0.8297872340425532' '0.871508379888268' '0.8832887700534761' 'best' '4'
 '28']
##################################################
233.csv
**************************************************
2022-02-13 22:25:26.912227
2022-02-13 22:25:28.658583
2022-02-13 22:25:36.034820
['233.csv' '77' '10' '11' '75' '0.8786127167630058' '0.8823529411764706'
 '0.872093023255814' '0.8771929824561403' '0.88

In [35]:
file_path = 'E:\datatest\PseDPC\d2\d24' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
241.csv
**************************************************
2022-02-13 22:38:38.082494
2022-02-13 22:38:39.626325
2022-02-13 22:38:42.428856
['241.csv' '70' '6' '18' '79' '0.861271676300578' '0.9294117647058824'
 '0.8144329896907216' '0.868131868131868' '0.8560160427807486' 'random'
 '6' '188']
##################################################
242.csv
**************************************************
2022-02-13 22:39:48.211853
2022-02-13 22:39:49.725804
2022-02-13 22:39:56.824781
['242.csv' '74' '9' '14' '76' '0.8670520231213873' '0.8941176470588236'
 '0.8444444444444444' '0.8685714285714287' '0.8710561497326204' 'best' '3'
 '47']
##################################################
243.csv
**************************************************
2022-02-13 22:41:52.478351
2022-02-13 22:41:54.078071
2022-02-13 22:42:01.836313
['243.csv' '75' '10' '13' '75' '0.8670520231213873' '0.8823529411764706'
 '0.8522727272727273' '0.8670520231213872' '0

In [43]:
file_path = 'E:\datatest\PseDPC\d2\d25' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
251.csv
**************************************************
2022-02-13 23:34:22.714092
2022-02-13 23:34:24.684819
2022-02-13 23:34:34.317050
['251.csv' '67' '19' '21' '66' '0.7687861271676301' '0.7764705882352941'
 '0.7586206896551724' '0.7674418604651162' '0.7764705882352941' 'best' '5'
 '210']
##################################################
252.csv
**************************************************
2022-02-13 23:34:39.960978
2022-02-13 23:34:41.812024
2022-02-13 23:34:50.306298
['252.csv' '74' '16' '14' '69' '0.8265895953757225' '0.8117647058823529'
 '0.8313253012048193' '0.8214285714285715' '0.8383689839572191' 'best' '3'
 '210']
##################################################
253.csv
**************************************************
2022-02-13 23:34:54.255704
2022-02-13 23:34:56.130688
2022-02-13 23:35:04.443477
['253.csv' '69' '20' '19' '65' '0.7745664739884393' '0.7647058823529411'
 '0.7738095238095238' '0.7692307692307693'

In [36]:
file_path = 'E:\datatest\PseDPC\d2\d26' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
261.csv
**************************************************
2022-02-13 22:56:32.487917
2022-02-13 22:56:34.128557
2022-02-13 22:56:41.728228
['261.csv' '78' '16' '10' '69' '0.8497109826589595' '0.8117647058823529'
 '0.8734177215189873' '0.8414634146341463' '0.845855614973262' 'best' '5'
 '82']
##################################################
262.csv
**************************************************
2022-02-13 22:59:11.407766
2022-02-13 22:59:13.134147
2022-02-13 22:59:21.443914
['262.csv' '74' '8' '14' '77' '0.8728323699421965' '0.9058823529411765'
 '0.8461538461538461' '0.8749999999999999' '0.8786096256684492' 'best' '3'
 '124']
##################################################
263.csv
**************************************************
2022-02-13 23:01:19.166948
2022-02-13 23:01:20.815536
2022-02-13 23:01:28.509983
['263.csv' '73' '8' '15' '77' '0.8670520231213873' '0.9058823529411765'
 '0.8369565217391305' '0.8700564971751413' '0.

In [44]:
file_path = 'E:\datatest\PseDPC\d2\d27' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
271.csv
**************************************************
2022-02-13 23:36:31.115556
2022-02-13 23:36:32.884823
2022-02-13 23:36:40.953269
['271.csv' '72' '22' '16' '63' '0.7803468208092486' '0.7411764705882353'
 '0.7974683544303798' '0.7682926829268293' '0.8098262032085561' 'best' '5'
 '210']
##################################################
272.csv
**************************************************
2022-02-13 23:36:46.177259
2022-02-13 23:36:48.386347
2022-02-13 23:36:58.446462
['272.csv' '70' '10' '18' '75' '0.838150289017341' '0.8823529411764706'
 '0.8064516129032258' '0.8426966292134832' '0.7981951871657754' 'best' '3'
 '160']
##################################################
273.csv
**************************************************
2022-02-13 23:37:02.800782
2022-02-13 23:37:04.908146
2022-02-13 23:37:14.173369
['273.csv' '73' '19' '15' '66' '0.8034682080924855' '0.7764705882352941'
 '0.8148148148148148' '0.7951807228915663' 

In [45]:
file_path = 'E:\datatest\PseDPC\d2\d28' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
281.csv
**************************************************
2022-02-13 23:38:30.623862
2022-02-13 23:38:32.277415
2022-02-13 23:38:39.949860
['281.csv' '77' '19' '11' '66' '0.8265895953757225' '0.7764705882352941'
 '0.8571428571428571' '0.8148148148148148' '0.8432486631016043' 'best' '3'
 '160']
##################################################
282.csv
**************************************************
2022-02-13 23:38:43.313881
2022-02-13 23:38:45.066174
2022-02-13 23:38:53.370091
['282.csv' '69' '11' '19' '74' '0.8265895953757225' '0.8705882352941177'
 '0.7956989247311828' '0.8314606741573034' '0.867312834224599' 'best' '3'
 '160']
##################################################
283.csv
**************************************************
2022-02-13 23:38:56.905495
2022-02-13 23:38:58.447371
2022-02-13 23:39:06.793042
['283.csv' '80' '11' '8' '74' '0.8901734104046243' '0.8705882352941177'
 '0.9024390243902439' '0.8862275449101796' '

In [46]:
file_path = 'E:\datatest\PseDPC\d2\d29' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
291.csv
**************************************************
2022-02-13 23:39:58.982410
2022-02-13 23:40:01.027937
2022-02-13 23:40:10.910496
['291.csv' '74' '6' '14' '79' '0.884393063583815' '0.9294117647058824'
 '0.8494623655913979' '0.8876404494382023' '0.8628342245989304' 'best' '3'
 '160']
##################################################
292.csv
**************************************************
2022-02-13 23:40:14.617605
2022-02-13 23:40:16.872544
2022-02-13 23:40:26.934650
['292.csv' '71' '14' '17' '71' '0.8208092485549133' '0.8352941176470589'
 '0.8068181818181818' '0.8208092485549133' '0.8302139037433156' 'best' '6'
 '10']
##################################################
293.csv
**************************************************
2022-02-13 23:40:31.509385
2022-02-13 23:40:33.328546
2022-02-13 23:40:42.025348
['293.csv' '77' '17' '11' '68' '0.838150289017341' '0.8'
 '0.8607594936708861' '0.829268292682927' '0.7899732620320856

In [47]:
file_path = 'E:\datatest\PseDPC\d2\d210' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2101.csv
**************************************************
2022-02-13 23:41:44.280684
2022-02-13 23:41:46.352170
2022-02-13 23:41:56.379342
['2101.csv' '70' '10' '18' '75' '0.838150289017341' '0.8823529411764706'
 '0.8064516129032258' '0.8426966292134832' '0.7671791443850269' 'best' '3'
 '110']
##################################################
2102.csv
**************************************************
2022-02-13 23:42:00.347725
2022-02-13 23:42:01.928467
2022-02-13 23:42:09.561046
['2102.csv' '64' '14' '24' '71' '0.7803468208092486' '0.8352941176470589'
 '0.7473684210526316' '0.7888888888888889' '0.7579545454545455' 'best' '3'
 '110']
##################################################
2103.csv
**************************************************
2022-02-13 23:42:13.003836
2022-02-13 23:42:14.639459
2022-02-13 23:42:17.467892
['2103.csv' '63' '18' '25' '67' '0.7514450867052023' '0.788235294117647'
 '0.7282608695652174' '0.7570621468926

In [48]:
file_path = 'E:\datatest\PseDPC\d2\d211' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2111.csv
**************************************************
2022-02-13 23:43:26.736564
2022-02-13 23:43:28.347254
2022-02-13 23:43:35.758456
['2111.csv' '74' '13' '14' '72' '0.8439306358381503' '0.8470588235294118'
 '0.8372093023255814' '0.8421052631578947' '0.7633021390374332' 'best' '3'
 '110']
##################################################
2112.csv
**************************************************
2022-02-13 23:43:39.080571
2022-02-13 23:43:41.002396
2022-02-13 23:43:50.024458
['2112.csv' '71' '11' '17' '74' '0.838150289017341' '0.8705882352941177'
 '0.8131868131868132' '0.8409090909090909' '0.7877005347593583' 'best' '3'
 '110']
##################################################
2113.csv
**************************************************
2022-02-13 23:43:53.371303
2022-02-13 23:43:55.009946
2022-02-13 23:44:02.905793
['2113.csv' '68' '23' '20' '62' '0.7514450867052023' '0.7294117647058823'
 '0.7560975609756098' '0.742514970059

In [49]:
file_path = 'E:\datatest\PseDPC\d2\d212' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2121.csv
**************************************************
2022-02-13 23:45:18.164440
2022-02-13 23:45:19.228620
2022-02-13 23:45:21.377870
['2121.csv' '69' '30' '19' '55' '0.7167630057803468' '0.6470588235294118'
 '0.7432432432432432' '0.6918238993710691' '0.7127005347593585' 'random'
 '3' '110']
##################################################
2122.csv
**************************************************
2022-02-13 23:45:22.206655
2022-02-13 23:45:23.251858
2022-02-13 23:45:25.665383
['2122.csv' '68' '35' '20' '50' '0.6820809248554913' '0.5882352941176471'
 '0.7142857142857143' '0.6451612903225806' '0.6105614973262032' 'random'
 '4' '10']
##################################################
2123.csv
**************************************************
2022-02-13 23:45:26.935983
2022-02-13 23:45:28.885755
2022-02-13 23:45:39.524293
['2123.csv' '69' '18' '19' '67' '0.7861271676300579' '0.788235294117647'
 '0.7790697674418605' '0.783625730

In [50]:
file_path = 'E:\datatest\PseDPC\d2\d213' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2131.csv
**************************************************
2022-02-13 23:46:35.894473
2022-02-13 23:46:37.684683
2022-02-13 23:46:47.440581
['2131.csv' '71' '11' '17' '74' '0.838150289017341' '0.8705882352941177'
 '0.8131868131868132' '0.8409090909090909' '0.7804812834224599' 'best' '3'
 '210']
##################################################
2132.csv
**************************************************
2022-02-13 23:46:51.158661
2022-02-13 23:46:53.062540
2022-02-13 23:46:55.959822
['2132.csv' '61' '10' '27' '75' '0.7861271676300579' '0.8823529411764706'
 '0.7352941176470589' '0.8021390374331552' '0.7724598930481285' 'random'
 '3' '110']
##################################################
2133.csv
**************************************************
2022-02-13 23:46:57.565494
2022-02-13 23:47:00.047850
2022-02-13 23:47:10.841999
['2133.csv' '74' '14' '14' '71' '0.838150289017341' '0.8352941176470589'
 '0.8352941176470589' '0.83529411764

In [51]:
file_path = 'E:\datatest\PseDPC\d2\d214' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2141.csv
**************************************************
2022-02-13 23:48:31.205538
2022-02-13 23:48:33.297928
2022-02-13 23:48:43.282214
['2141.csv' '74' '11' '14' '74' '0.8554913294797688' '0.8705882352941177'
 '0.8409090909090909' '0.8554913294797689' '0.8116978609625669' 'best' '3'
 '210']
##################################################
2142.csv
**************************************************
2022-02-13 23:48:47.296474
2022-02-13 23:48:49.327075
2022-02-13 23:48:59.770102
['2142.csv' '83' '25' '5' '60' '0.8265895953757225' '0.7058823529411765'
 '0.9230769230769231' '0.8000000000000002' '0.8475267379679146' 'best' '3'
 '160']
##################################################
2143.csv
**************************************************
2022-02-13 23:49:03.495135
2022-02-13 23:49:05.606508
2022-02-13 23:49:15.391307
['2143.csv' '72' '15' '16' '70' '0.8208092485549133' '0.8235294117647058'
 '0.813953488372093' '0.8187134502923

In [52]:
file_path = 'E:\datatest\PseDPC\d2\d215' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2151.csv
**************************************************
2022-02-13 23:50:54.873177
2022-02-13 23:50:56.724191
2022-02-13 23:51:05.632358
['2151.csv' '80' '20' '8' '65' '0.838150289017341' '0.7647058823529411'
 '0.8904109589041096' '0.8227848101265822' '0.7676470588235293' 'best' '3'
 '60']
##################################################
2152.csv
**************************************************
2022-02-13 23:51:09.624681
2022-02-13 23:51:11.615350
2022-02-13 23:51:19.409414
['2152.csv' '78' '21' '10' '64' '0.8208092485549133' '0.7529411764705882'
 '0.8648648648648649' '0.8050314465408804' '0.7546791443850267' 'best' '3'
 '60']
##################################################
2153.csv
**************************************************
2022-02-13 23:51:22.892097
2022-02-13 23:51:24.612515
2022-02-13 23:51:33.721127
['2153.csv' '66' '14' '22' '71' '0.791907514450867' '0.8352941176470589'
 '0.7634408602150538' '0.7977528089887641

In [53]:
file_path = 'E:\datatest\PseDPC\d2\d216' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2161.csv
**************************************************
2022-02-13 23:52:52.724783
2022-02-13 23:52:54.499030
2022-02-13 23:53:03.222663
['2161.csv' '73' '14' '15' '71' '0.8323699421965318' '0.8352941176470589'
 '0.8255813953488372' '0.8304093567251463' '0.8084224598930483' 'best' '3'
 '210']
##################################################
2162.csv
**************************************************
2022-02-13 23:53:07.223957
2022-02-13 23:53:08.988258
2022-02-13 23:53:18.158701
['2162.csv' '76' '14' '12' '71' '0.8497109826589595' '0.8352941176470589'
 '0.8554216867469879' '0.8452380952380952' '0.7925133689839572' 'best' '3'
 '210']
##################################################
2163.csv
**************************************************
2022-02-13 23:53:23.180800
2022-02-13 23:53:25.419809
2022-02-13 23:53:35.407088
['2163.csv' '77' '14' '11' '71' '0.8554913294797688' '0.8352941176470589'
 '0.8658536585365854' '0.85029940119

In [54]:
file_path = 'E:\datatest\PseDPC\d2\d217' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2171.csv
**************************************************
2022-02-13 23:55:17.905174
2022-02-13 23:55:19.660479
2022-02-13 23:55:28.003185
['2171.csv' '69' '8' '19' '77' '0.8439306358381503' '0.9058823529411765'
 '0.8020833333333334' '0.850828729281768' '0.7941176470588236' 'best' '3'
 '160']
##################################################
2172.csv
**************************************************
2022-02-13 23:55:31.676351
2022-02-13 23:55:33.466540
2022-02-13 23:55:42.203187
['2172.csv' '76' '6' '12' '79' '0.8959537572254336' '0.9294117647058824'
 '0.8681318681318682' '0.8977272727272727' '0.8677139037433155' 'best' '3'
 '160']
##################################################
2173.csv
**************************************************
2022-02-13 23:55:45.838461
2022-02-13 23:55:47.633664
2022-02-13 23:55:56.311452
['2173.csv' '76' '6' '12' '79' '0.8959537572254336' '0.9294117647058824'
 '0.8681318681318682' '0.897727272727272

In [55]:
file_path = 'E:\datatest\PseDPC\d2\d218' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2181.csv
**************************************************
2022-02-13 23:57:25.775059
2022-02-13 23:57:27.656027
2022-02-13 23:57:36.833473
['2181.csv' '74' '8' '14' '77' '0.8728323699421965' '0.9058823529411765'
 '0.8461538461538461' '0.8749999999999999' '0.8238636363636365' 'best' '3'
 '60']
##################################################
2182.csv
**************************************************
2022-02-13 23:57:40.184507
2022-02-13 23:57:41.959785
2022-02-13 23:57:50.725305
['2182.csv' '78' '10' '10' '75' '0.884393063583815' '0.8823529411764706'
 '0.8823529411764706' '0.8823529411764706' '0.8602272727272726' 'best' '3'
 '160']
##################################################
2183.csv
**************************************************
2022-02-13 23:57:54.373545
2022-02-13 23:57:56.180709
2022-02-13 23:58:04.934289
['2183.csv' '78' '10' '10' '75' '0.884393063583815' '0.8823529411764706'
 '0.8823529411764706' '0.882352941176470

In [56]:
file_path = 'E:\datatest\PseDPC\d2\d219' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2191.csv
**************************************************
2022-02-13 23:59:32.372349
2022-02-13 23:59:34.131670
2022-02-13 23:59:42.771526
['2191.csv' '73' '11' '15' '74' '0.8497109826589595' '0.8705882352941177'
 '0.8314606741573034' '0.8505747126436781' '0.7831550802139038' 'best' '3'
 '60']
##################################################
2192.csv
**************************************************
2022-02-13 23:59:45.935062
2022-02-13 23:59:47.596645
2022-02-13 23:59:55.793713
['2192.csv' '72' '12' '16' '73' '0.838150289017341' '0.8588235294117647'
 '0.8202247191011236' '0.8390804597701149' '0.7936497326203209' 'best' '3'
 '60']
##################################################
2193.csv
**************************************************
2022-02-13 23:59:58.957248
2022-02-14 00:00:00.678616
2022-02-14 00:00:09.200815
['2193.csv' '76' '11' '12' '74' '0.8670520231213873' '0.8705882352941177'
 '0.8604651162790697' '0.86549707602339

In [57]:
file_path = 'E:\datatest\PseDPC\d2\d220' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
2201.csv
**************************************************
2022-02-14 00:01:26.295548
2022-02-14 00:01:28.506633
2022-02-14 00:01:32.187783
['2201.csv' '61' '21' '27' '64' '0.7225433526011561' '0.7529411764705882'
 '0.7032967032967034' '0.7272727272727272' '0.6405080213903743' 'random'
 '7' '110']
##################################################
2202.csv
**************************************************
2022-02-14 00:01:34.836696
2022-02-14 00:01:37.089668
2022-02-14 00:01:46.428713
['2202.csv' '68' '14' '20' '71' '0.8034682080924855' '0.8352941176470589'
 '0.7802197802197802' '0.8068181818181818' '0.752807486631016' 'best' '4'
 '160']
##################################################
2203.csv
**************************************************
2022-02-14 00:01:50.689319
2022-02-14 00:01:52.398709
2022-02-14 00:02:01.770635
['2203.csv' '72' '15' '16' '70' '0.8208092485549133' '0.8235294117647058'
 '0.813953488372093' '0.81871345029